# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("19-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'wubu, brr, uru, uw, gug, rgrur, ub, rrwg, wrw, uugg, wwb, gwrg, rgb, bbrbgw, guwb, rub, wgww, wgu, bwbu, bgubwb, gbuu, buwr, uub, wubgr, uwugbwb, gwwbbwrr, rbgbgr, gbuuww, bur, gubgbr, wgug, rrb, wbrugwrb, brb, gbu, rrwrbgrr, uburgr, bbrgrwg, rbbb, gbr, grgug, rr, wrbw, uugbgb, bbg, br, rubbbu, bbggubb, brwwr, rgwbrr, ubr, wgugg, grrburw, r, uuwrwug, gguw, ww, bbrb, bguw, uurr, bru, ru, bwubwrb, wur, wrbgru, bbug, gbbrw, rwbu, wrwrr, bgu, wrgbur, ubugg, wb, rguw, ubbrgbw, buwurgb, wrwruuu, rgwwu, wwrbw, uwwwubb, wg, wrgwrwur, gwg, wbw, wgrg, ruurruu, uwu, bwwbuuug, rbr, urgbg, gugu, gwgwbgur, gbg, gbwu, uur, ggwbb, grub, gwrrw, uuu, rwr, urwbu, uwwbrwgg, gbb, guwg, urguu, urwgug, grwuwu, rwb, rbwwbg, grbuwu, bwbbbb, rbgr, rwrrur, gur, rwruwg, uuwrw, wbb, uug, rrbr, wrbbb, ggwgg, wrrur, bub, ug, guubgw, gwubb, wub, rruw, uruwwww, rrugwb, urbu, wbbu, bggr, urgr, brww, bwg, rggwgg, urr, bbbgwbbw, bwrgw, ugb, gwr, uwugwgu, bbgrubb, wbr, brg, ggg, ububww, grw, grr, wrb, gbuubg, gbuuu, b, u

In [4]:
test_data_raw = r"""r, wr, b, g, bwu, rb, gb, br

brwrr
bggr
gbbr
rrbgbr
ubwu
bwurrg
brgr
bbrgwb"""






def preprocess_data (data):
    towels, patterns = data.split("\n\n") 
    towels = [row for row in towels.split(", ")]
    patterns = patterns.split("\n")

    return towels, patterns


test_data = preprocess_data(test_data_raw)

display(test_data)


(['r', 'wr', 'b', 'g', 'bwu', 'rb', 'gb', 'br'],
 ['brwrr', 'bggr', 'gbbr', 'rrbgbr', 'ubwu', 'bwurrg', 'brgr', 'bbrgwb'])

In [5]:
full_data = preprocess_data(data_raw)
full_data

(['wubu',
  'brr',
  'uru',
  'uw',
  'gug',
  'rgrur',
  'ub',
  'rrwg',
  'wrw',
  'uugg',
  'wwb',
  'gwrg',
  'rgb',
  'bbrbgw',
  'guwb',
  'rub',
  'wgww',
  'wgu',
  'bwbu',
  'bgubwb',
  'gbuu',
  'buwr',
  'uub',
  'wubgr',
  'uwugbwb',
  'gwwbbwrr',
  'rbgbgr',
  'gbuuww',
  'bur',
  'gubgbr',
  'wgug',
  'rrb',
  'wbrugwrb',
  'brb',
  'gbu',
  'rrwrbgrr',
  'uburgr',
  'bbrgrwg',
  'rbbb',
  'gbr',
  'grgug',
  'rr',
  'wrbw',
  'uugbgb',
  'bbg',
  'br',
  'rubbbu',
  'bbggubb',
  'brwwr',
  'rgwbrr',
  'ubr',
  'wgugg',
  'grrburw',
  'r',
  'uuwrwug',
  'gguw',
  'ww',
  'bbrb',
  'bguw',
  'uurr',
  'bru',
  'ru',
  'bwubwrb',
  'wur',
  'wrbgru',
  'bbug',
  'gbbrw',
  'rwbu',
  'wrwrr',
  'bgu',
  'wrgbur',
  'ubugg',
  'wb',
  'rguw',
  'ubbrgbw',
  'buwurgb',
  'wrwruuu',
  'rgwwu',
  'wwrbw',
  'uwwwubb',
  'wg',
  'wrgwrwur',
  'gwg',
  'wbw',
  'wgrg',
  'ruurruu',
  'uwu',
  'bwwbuuug',
  'rbr',
  'urgbg',
  'gugu',
  'gwgwbgur',
  'gbg',
  'gbwu',
  'uur',
  'g

In [26]:

def solution (data, verbose=False):
    towels, patterns = data
    total = 0

    def next_towels(pattern, towels):
        return [pattern[len(towel):] for towel in towels if towel == pattern[:len(towel)]]

    for pattern in patterns:
        tree = {}

        def build_tree(root, tree, level):
            
            if root != "":
                next_patterns = next_towels(root, towels)
                tree[(root, level)] = next_patterns
                for next_pattern in next_patterns:
                    if (next_pattern, level+1) not in tree:
                        build_tree(next_pattern, tree, level+1)
                
        build_tree(pattern, tree, 0)

        if sum([1 for value in  tree.values() if "" in  value ]) > 0:
            total += 1
    return total

sol = solution(test_data, verbose=True)
print(sol)


6


In [27]:
sol = solution(full_data)
print(sol)

258


# Part 2

In [30]:

def solution2 (data, verbose=False):
    towels, patterns = data
    total = 0

    def next_towels(pattern, towels):
        return [pattern[len(towel):] for towel in towels if towel == pattern[:len(towel)]]

    for pattern in patterns:

        # print(pattern)
        @cache
        def build_tree(root):
            
            if root == "":
                return 1
            else:
                next_patterns = next_towels(root, towels)
                ways = 0
                for next_pattern in next_patterns:
                    ways += build_tree(next_pattern)
                return ways
                
        ways = build_tree(pattern)
        total += ways

    return total


sol = solution2(test_data, verbose=True)
print(sol)

16


In [31]:
sol = solution2(full_data)
print(sol)

632423618484345
